In [57]:
#import all required libraries
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time
import warnings
warnings.filterwarnings("ignore")

from selenium.common.exceptions import NoSuchElementException

### Scrape the Car prices and other details of 5000 used cars from cartrade.com

In [498]:
#connecting to webdriver
driver = webdriver.Chrome('chromedriver')

In [81]:
url = 'https://www.cartrade.com/'
driver.get(url)

In [82]:
#Finding web element for 'All used cars' tab
search_tab = driver.find_element(By.XPATH,"/html/body/section[1]/div/div/div/div[1]/div/div[1]/div[2]/div[2]")
search_tab.click()

In [83]:
url = []
loc = []
fuel =[]
price = []
km = []

In [90]:
#Fetching url to open each description page as well as information of used cars on main page
a=1
start=0
end=200

for page in range(start,end):
    
    #extracting url tags
    url_tags= driver.find_elements(By.XPATH, '//h2[@class="h2heading truncate"]/a')
    for i in url_tags:
        url.append(i.get_attribute('href'))
        a = a+1
    
    
    #extracting location tags
    loc_tags = driver.find_elements(By.XPATH, '//div[@class="info_cr_new"]')
    for i in loc_tags:
        loc.append(i.text.split('\n|\n')[2])
    
    
    #extracting fuel type tags
    fuel_tags = driver.find_elements(By.XPATH, '//div[@class="info_cr_new"]')
    for i in fuel_tags:
        fuel.append(i.text.split('\n|\n')[1])
    
    
    #extracting car price tags
    price_tags = driver.find_elements(By.XPATH, '//div[@class="cr_prc"]')
    for i in price_tags:
        price.append(i.text)
    
    
    #extracting driven kilometers tags
    km_tags = driver.find_elements(By.XPATH, '//div[@class="info_cr_new"]')
    for i in km_tags:
        km.append(i.text.split('\n|\n')[0])
    
    if a>=10000:
        break
    
    time.sleep(5)
    #For pressing next button
    try:
        button = driver.find_element(By.XPATH,'//li[@class="next"]')
        button.click()
    except:
        break # In case number of pages end before reaching 10000
    time.sleep(5)

In [91]:
len(url)

8318

In [92]:
len(loc)

8318

In [93]:
len(fuel)

8318

In [94]:
len(price)

8318

In [95]:
len(km)

8318

In [297]:
brand = []
model = []
var = []
col = []
year = []
own = []

In [960]:
#let's extract all web elements having required parent tags and then extract required elements from them
for i in url[0:5000]:
    driver.get(i)
    time.sleep(3)
    driver.execute_script("window.stop();")
    
    #Extracting Brand from parent tag
    try:
        brand_test = []
        brand_tag = driver.find_element(By.XPATH,'//div[@class="wrapper v_content"]')
        brand_test.append(brand_tag.text.split('\n')[3])
        brand.append(brand_test[0].split("»")[0])
    except:
        brand.append(None)
    
    
    #Extracting Model name from parent tag
    try:
        model_test = []
        model_tag = driver.find_element(By.XPATH,'//div[@class="wrapper v_content"]')
        model_test.append(model_tag.text.split('\n')[4])
        model.append(model_test[0].split("»")[0])
    except:
        model.append(None)
    
    
    #Extracting Variant from parent tag
    try:
        var_tag = driver.find_element(By.XPATH,'//div[@class="wrapper v_content"]')
        var.append(var_tag.text.split('\n')[5])
    except:
        var.append(None)
            
        
    #Extracting Manufacturing year from parent tag
    try:
        year_test=[]
        year_tag = driver.find_element(By.XPATH,'//table[@class = "v_table"]')
        year_test.append(year_tag.text.split('\n'))
        
        # to get string with substring(MANUFACTURING YEAR) in it
        year_try = [i for i in year_test[0] if "MANUFACTURING YEAR" in i]
        year.append(year_try[0].split(" ")[2])
    except:
        year.append(None)
            
        
    #Extracting Number of owners from parent tag
    try:
        own_test=[]
        own_tag = driver.find_element(By.XPATH,'//table[@class = "v_table"]')
        own_test.append(own_tag.text.split('\n'))
        
        # to get string with substring(NUMBER OF OWNERS) in it
        own_try = [i for i in own_test[0] if "NUMBER OF OWNERS" in i]
        own.append(own_try[0].split(" ")[3])
    except:
        own.append(None)
    
    time.sleep(3)

In [961]:
len(brand)

5000

In [962]:
len(model)

5000

In [963]:
len(var)

5000

In [964]:
len(year)

5000

In [965]:
len(own)

5000

In [971]:
# Editing price variable data
fprice = [j.replace("₹"," ").split(" ")[1] for j in price]

In [973]:
car = pd.DataFrame()
car['Brand'] = brand
car['Model'] = model
car['Variant'] = var
car['Manufacturing Year'] = year
car['Driven kilometers'] = km[0:5000]
car['Fuel Type'] = fuel[0:5000]
car['Number of owners'] = own
car['Location'] = loc[0:5000]
car['Price (in ₹)'] = fprice[0:5000]
car

,Brand,Model,Variant,Manufacturing Year,Driven kilometers,Fuel Type,Number of owners,Location,Price (in ₹)
0,Hyundai,Verna,1.6 VTVT SX,2017,"64,685 KMs",Petrol,First,Faridabad,"8,25,000"
1,Hyundai,i10,Sportz 1.2 AT,2010,"38,798 KMs",Petrol,Second,Mumbai,"2,95,000"
2,Skoda,Rapid,1.6 MPI Active,2020,"19,922 KMs",Petrol,Second,Coimbatore,"8,75,000"
3,Toyota,Innova,2.5 V 8 STR,2009,"2,75,000 KMs",Diesel,First,Hyderabad,"6,65,000"
4,Maruti Suzuki,Wagon R,LXI ABS,2016,"49,679 KMs",Petrol,First,Faridabad,"3,45,000"
...,...,...,...,...,...,...,...,...,...
4995,Jeep,Compass,Limited Plus Petrol AT,2020,"23,500 KMs",Petrol,First,Delhi,"21,50,000"
4996,Skoda,Kodiaq,Style 2.0 TDI 4x4 AT,2017,"1,30,000 KMs",Diesel,First,Ahmedabad,"25,00,000"
4997,None,None,None,None,"55,740 KMs",Diesel,None,Coimbatore,"4,95,000"
4998,Maruti Suzuki,Swift DZire,VDI,2013,"95,000 KMs",Diesel,Second,Ahmedabad,"4,50,000"


In [974]:
# saving the excel
car.to_excel('car_price_data.xlsx')

In [970]:
driver.close()